In [4]:
import statsmodels.formula.api as smf
from differences import ATTgt
from differences.did.pscore_cal import pscore_mle
from matplotlib import pyplot as plt

import figure_and_table_constants
import figure_utilities
from analysis_utilities import produce_summary_statistics, aggregate_by_time_and_plot, aggregate_by_event_time_and_plot

plt.rcParams["figure.dpi"] = 300
plt.rcParams['savefig.dpi'] = 300
import numpy as np
import os
import pandas as pd

# Store paths.
INPUT_DATA = "/Users/arjunshanmugam/Documents/GitHub/seniorthesis/data/03_cleaned/crime_analysis.csv"
OUTPUT_FIGURES = "/Users/arjunshanmugam/Documents/GitHub/seniorthesis/output/DiD/figures"
OUTPUT_TABLES = "/Users/arjunshanmugam/Documents/GitHub/seniorthesis/output/DiD/tables"

# Read restricted sample panel dataset into memory.
df = pd.read_csv(INPUT_DATA)

In [5]:
# Store list of crime variable names and create dictionaries which map between month variable names to integers.
years = [str(year) for year in range(2013, 2023)]
months = ["0" + str(month) for month in range(1, 10)] + [str(month) for month in range(10, 13)]
value_vars = ["2012-12"] + [str(year) + "-" + str(month) for year in years for month in months]
value_vars_zestimates = [value_var + "_crime" for value_var in value_vars]
name_change_dictionary = {value_vars_zestimate: value_var for value_vars_zestimate, value_var in zip(value_vars_zestimates, value_vars)}
df = df.rename(columns=name_change_dictionary)
month_to_int_dictionary = {key: value + 1 for value, key in enumerate(value_vars)}
int_to_month_dictionary = {key + 1: value for key, value in enumerate(value_vars)}

In [6]:
print(df[value_vars])

KeyError: "None of [Index(['2012-12', '2013-01', '2013-02', '2013-03', '2013-04', '2013-05',\n       '2013-06', '2013-07', '2013-08', '2013-09',\n       ...\n       '2022-03', '2022-04', '2022-05', '2022-06', '2022-07', '2022-08',\n       '2022-09', '2022-10', '2022-11', '2022-12'],\n      dtype='object', length=121)] are in the [columns]"